<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=ba6632db15fb9137340146b6ea16e72d14097dd5615d1aa0886dc5f7f7df0911
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7


Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-03 09:42:49
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.41 C
-------------------
Today PnL: 28.70 K (0.2%)
Current PnL: -21.08 L (-13.99%)
CY Booked + Current PnL: -9.64 L (-6.4%)
-------------------
Total profit:  1.33 L
Total loss:  -22.41 L
-------------------
Total Booked + Current PnL: 17.34 L (14.02%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.14%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 88.39 L (62.91%)
Deployed:  1.24 C
Current:  1.41 C
CAGR/XIRR %: 7.77%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.83,65.0,H-MC,12.86,114379.0,13875.0,15235.0,0.61,13.81,13.32,28.96,79.0,0.91,0.82,52.46,MH,ATH,METALS
77,TTKPRESTIG,770.00,94.45,40.0,M-SC,1.70,83788.0,-16989.0,17084.0,0.40,-16.86,20.39,0.09,245.0,-0.99,0.60,10.08,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,8.56,44.0,H-MC,2.92,187450.0,22780.0,20545.0,-0.18,13.83,10.96,26.31,89.0,1.11,1.34,22.92,X40N,NTT,AC
42,ITC,452.00,-38.77,47.0,H-LC,2.16,197996.0,-2142.0,23027.0,-0.17,-1.07,11.63,10.44,4.0,-0.09,1.41,4.43,X40,NTT,FMCG
34,ICICIGI,2260.25,-17.56,57.0,H-MC,1.76,141902.0,5769.0,25358.0,0.74,4.24,17.87,22.86,91.0,0.23,1.01,17.39,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.0,-188.64,81.0,M-SC,8.40,152622.0,3402.0,140779.0,5.42,2.28,92.24,96.62,208.0,0.02,1.09,65.17,XY25,NTT,FINANCE
69,SURYODAY,240.0,75.44,82.0,H-SC,3.97,161527.0,-17544.0,83267.0,1.78,-9.80,51.55,36.71,135.0,-0.21,1.15,60.27,XR,NTT,BANKS
80,UNITDSPR,1644.0,-11.66,56.0,H-LC,7.19,231632.0,-2512.0,51144.0,1.69,-1.07,22.08,20.77,86.0,-0.05,1.66,5.77,X40N,NTT,BREWERIES
36,IEX,219.0,-32.01,49.0,H-SC,17.62,194526.0,-5428.0,102651.0,1.54,-2.71,52.77,48.63,115.0,-0.05,1.39,8.52,XR,NTT,MISC
82,VALIANTORG,838.0,-307.11,42.0,H-SC,3.74,115260.0,-48345.0,169663.0,1.50,-29.55,147.20,74.15,139.0,-0.28,0.82,47.74,XR,NTT,CHEMICALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,GREENPANEL,537.00,214.60,51.0,M-SC,2.09,140593.0,-42485.0,121458.0,-3.09,-23.21,86.39,43.14,230.0,-0.35,1.00,32.25,XY24,NTT,MISC
49,MASFIN,398.61,-20.05,41.0,H-SC,11.55,90765.0,-7215.0,28818.0,-1.40,-7.36,31.75,22.05,152.0,-0.25,0.65,31.93,XR,ATH,FINANCE
86,WIPRO,420.00,-15.61,38.0,M-LC,5.89,148006.0,-2939.0,112396.0,-0.98,-1.95,75.94,72.51,53.0,-0.03,1.06,3.96,XR,NTT,IT
8,AWL,485.00,-65.02,51.0,H-MC,3.04,249255.0,-52496.0,213911.0,-0.78,-17.40,85.82,53.50,116.0,-0.25,1.78,10.71,XY24,NTT,FMCG
70,SYMPHONY,1306.00,-44.18,36.0,M-SC,17.94,118817.0,-52324.0,52268.0,-0.60,-30.57,43.99,-0.03,196.0,-1.00,0.85,1.44,OX40N,NTT,DURABLES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.0,-21.34,47.0,H-MC,2.18,135272.0,-516.0,43274.0,0.33,-0.38,31.99,31.48,107.0,-0.01,0.97,11.69,X40,ATH,INSURANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,105.37,64.0,H-SC,12.08,129816.0,-12663.0,32181.0,-0.31,-8.89,24.79,13.70,163.0,-0.39,0.93,53.22,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.42,56.0,H-SC,1.82,224298.0,-45369.0,81712.0,0.00,-16.82,36.43,13.47,138.0,-0.56,1.60,13.45,XY24,NTT,PAINTS
77,TTKPRESTIG,770.0,94.45,40.0,M-SC,1.70,83788.0,-16989.0,17084.0,0.40,-16.86,20.39,0.09,245.0,-0.99,0.60,10.08,OX40N,NTT,DURABLES
18,CERA,9475.0,-22.43,39.0,H-SC,1.48,141289.0,-34614.0,76635.0,0.61,-19.68,54.24,23.89,149.0,-0.45,1.01,22.02,OX40N,NTT,CERAMICS
66,SIS,528.0,2095.85,44.0,H-SC,3.72,88625.0,-22407.0,46014.0,-0.44,-20.18,51.92,21.26,156.0,-0.49,0.63,19.74,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1246.09,69.0,M-SC,18.27,182987.0,1844.0,43624.0,-0.04,1.02,23.84,25.10,235.0,0.04,1.31,46.65,XY24,NTT,HEALTHCARE
60,SAIL,228.00,47.55,60.0,M-MC,11.90,232883.0,7921.0,159734.0,0.24,3.52,68.59,74.53,192.0,0.05,1.66,36.91,XY24,BTT,STEEL
32,HINDZINC,730.22,28.52,64.0,M-LC,3.86,209387.0,4311.0,108253.0,-0.23,2.10,51.70,54.89,52.0,0.04,1.50,26.70,X5K,ATH,METALS
14,BLUESTARCO,2080.00,8.56,44.0,H-MC,2.92,187450.0,22780.0,20545.0,-0.18,13.83,10.96,26.31,89.0,1.11,1.34,22.92,X40N,NTT,AC
84,VOLTAS,1530.00,-3.04,44.0,H-MC,2.21,203460.0,11718.0,26043.0,0.21,6.11,12.80,19.69,99.0,0.45,1.45,13.32,XY25,NTT,AC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.83,65.0,H-MC,12.86,114379.0,13875.0,15235.0,0.61,13.81,13.32,28.96,79.0,0.91,0.82,52.46,MH,ATH,METALS
11,BANKINDIA,190.00,-25.47,66.0,H-MC,10.01,191266.0,11458.0,100950.0,-0.22,6.37,52.78,62.52,88.0,0.11,1.37,42.63,XR,NTT,BANKS
32,HINDZINC,730.22,28.52,64.0,M-LC,3.86,209387.0,4311.0,108253.0,-0.23,2.10,51.70,54.89,52.0,0.04,1.50,26.70,X5K,ATH,METALS
37,INDIAMART,4850.92,-54.27,39.0,H-SC,9.41,127539.0,4203.0,129554.0,0.31,3.41,101.58,108.45,119.0,0.03,0.91,26.52,AR,ATH,MISC
86,WIPRO,420.00,-15.61,38.0,M-LC,5.89,148006.0,-2939.0,112396.0,-0.98,-1.95,75.94,72.51,53.0,-0.03,1.06,3.96,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-18.16,31.0,H-LC,5.81,206633.0,-45135.0,98337.0,0.53,-17.93,47.59,21.13,27.0,-0.46,1.48,11.15,X40,ATH,PAINTS
83,VBL,671.64,-22.00,31.0,H-LC,9.56,281167.0,-34675.0,147332.0,-0.51,-10.98,52.40,35.67,5.0,-0.24,2.01,1.35,X40N,ATH,FMCG
74,TCS,4476.75,-31.26,32.0,H-LC,12.48,273333.0,-72623.0,147490.0,-0.22,-20.99,53.96,21.64,1.0,-0.49,1.95,0.67,X40,ATH,IT
30,HCLTECH,1943.91,-4.09,36.0,H-LC,10.28,219094.0,-22830.0,86104.0,0.43,-9.44,39.30,26.15,8.0,-0.27,1.57,3.53,X40,ATH,IT
44,JIOFIN,387.00,-4.96,38.0,H-LC,12.82,258301.0,-9064.0,74520.0,0.10,-3.39,28.85,24.48,48.0,-0.12,1.85,49.72,XY24,BTT,FINANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,-2.61,45.0,H-LC,1.77,156580.0,-29515.0,76990.0,0.32,-15.86,49.17,25.51,15.0,-0.38,1.12,15.39,AR,ATH,ELECTRICAL
48,LTIM,7230.2,-6.94,43.0,H-LC,1.79,230445.0,-15706.0,94920.0,-0.04,-6.38,41.19,32.18,16.0,-0.17,1.65,27.36,X200,ATH,IT
42,ITC,452.0,-38.77,47.0,H-LC,2.16,197996.0,-2142.0,23027.0,-0.17,-1.07,11.63,10.44,4.0,-0.09,1.41,4.43,X40,NTT,FMCG
57,RELIANCE,1533.0,-15.17,39.0,H-LC,3.54,212566.0,2200.0,26592.0,-0.45,1.05,12.51,13.68,37.0,0.08,1.52,17.74,XY25,NTT,REFINERIES
51,NESTLEIND,1377.0,-11.41,45.0,H-LC,3.78,272563.0,7137.0,49661.0,-0.14,2.69,18.22,21.40,11.0,0.14,1.95,9.97,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.26,32.0,H-LC,12.48,273333.0,-72623.0,147490.0,-0.22,-20.99,53.96,21.64,1.0,-0.49,1.95,0.67,X40,ATH,IT
83,VBL,671.64,-22.00,31.0,H-LC,9.56,281167.0,-34675.0,147332.0,-0.51,-10.98,52.40,35.67,5.0,-0.24,2.01,1.35,X40N,ATH,FMCG
30,HCLTECH,1943.91,-4.09,36.0,H-LC,10.28,219094.0,-22830.0,86104.0,0.43,-9.44,39.30,26.15,8.0,-0.27,1.57,3.53,X40,ATH,IT
40,INFY,2275.00,-21.39,38.0,H-LC,8.25,306315.0,-6573.0,178245.0,-0.53,-2.10,58.19,54.87,3.0,-0.04,2.19,4.41,X40,BTT,IT
42,ITC,452.00,-38.77,47.0,H-LC,2.16,197996.0,-2142.0,23027.0,-0.17,-1.07,11.63,10.44,4.0,-0.09,1.41,4.43,X40,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-2.61,45.0,H-LC,1.77,156580.0,-29515.0,76990.0,0.32,-15.86,49.17,25.51,15.0,-0.38,1.12,15.39,AR,ATH,ELECTRICAL
42,ITC,452.00,-38.77,47.0,H-LC,2.16,197996.0,-2142.0,23027.0,-0.17,-1.07,11.63,10.44,4.0,-0.09,1.41,4.43,X40,NTT,FMCG
5,ASIANPAINT,3465.66,-18.16,31.0,H-LC,5.81,206633.0,-45135.0,98337.0,0.53,-17.93,47.59,21.13,27.0,-0.46,1.48,11.15,X40,ATH,PAINTS
19,CIPLA,1795.00,-21.84,46.0,H-LC,3.91,208720.0,4220.0,37194.0,0.69,2.06,17.82,20.25,10.0,0.11,1.49,10.98,X40N,BTT,PHARMA
57,RELIANCE,1533.00,-15.17,39.0,H-LC,3.54,212566.0,2200.0,26592.0,-0.45,1.05,12.51,13.68,37.0,0.08,1.52,17.74,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.26,32.0,H-LC,12.48,273333.0,-72623.0,147490.0,-0.22,-20.99,53.96,21.64,1.0,-0.49,1.95,0.67,X40,ATH,IT
40,INFY,2275.00,-21.39,38.0,H-LC,8.25,306315.0,-6573.0,178245.0,-0.53,-2.10,58.19,54.87,3.0,-0.04,2.19,4.41,X40,BTT,IT
42,ITC,452.00,-38.77,47.0,H-LC,2.16,197996.0,-2142.0,23027.0,-0.17,-1.07,11.63,10.44,4.0,-0.09,1.41,4.43,X40,NTT,FMCG
83,VBL,671.64,-22.00,31.0,H-LC,9.56,281167.0,-34675.0,147332.0,-0.51,-10.98,52.40,35.67,5.0,-0.24,2.01,1.35,X40N,ATH,FMCG
1,ABB,7934.00,-40.45,45.0,H-LC,7.66,248976.0,-12643.0,131858.0,-0.23,-4.83,52.96,45.57,7.0,-0.10,1.78,5.94,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,45.0,L-SC,37.50,79910.0,-33639.0,73693.0,0.48,-29.62,92.22,35.27,268.0,-0.46,0.57,95.58,XR,NTT,HOTELS
49,MASFIN,398.61,-20.05,41.0,H-SC,11.55,90765.0,-7215.0,28818.0,-1.40,-7.36,31.75,22.05,152.0,-0.25,0.65,31.93,XR,ATH,FINANCE
50,NATIONALUM,244.55,-42.83,65.0,H-MC,12.86,114379.0,13875.0,15235.0,0.61,13.81,13.32,28.96,79.0,0.91,0.82,52.46,MH,ATH,METALS
78,UJJIVANSFB,60.00,105.37,64.0,H-SC,12.08,129816.0,-12663.0,32181.0,-0.31,-8.89,24.79,13.70,163.0,-0.39,0.93,53.22,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-188.64,81.0,M-SC,8.40,152622.0,3402.0,140779.0,5.42,2.28,92.24,96.62,208.0,0.02,1.09,65.17,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.0,1246.09,69.0,M-SC,18.27,182987.0,1844.0,43624.0,-0.04,1.02,23.84,25.10,235.0,0.04,1.31,46.65,XY24,NTT,HEALTHCARE
69,SURYODAY,240.0,75.44,82.0,H-SC,3.97,161527.0,-17544.0,83267.0,1.78,-9.80,51.55,36.71,135.0,-0.21,1.15,60.27,XR,NTT,BANKS
61,SAMMAANCAP,326.0,-188.64,81.0,M-SC,8.40,152622.0,3402.0,140779.0,5.42,2.28,92.24,96.62,208.0,0.02,1.09,65.17,XY25,NTT,FINANCE
7,ATULAUTO,844.0,3591.89,50.0,M-SC,7.85,165733.0,-31399.0,121234.0,-0.56,-15.93,73.15,45.57,236.0,-0.26,1.18,17.50,XY24,NTT,AUTO
78,UJJIVANSFB,60.0,105.37,64.0,H-SC,12.08,129816.0,-12663.0,32181.0,-0.31,-8.89,24.79,13.70,163.0,-0.39,0.93,53.22,OX40N,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.98
1,25,43.09
2,50,74.86


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.29
LC    32.03
MC    23.70
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.94
X40      14.46
X40N     11.25
XR       11.12
XY25     10.43
AR        8.11
OX40N     7.77
X200      1.65
MH        1.63
X5K       1.50
SR        1.16
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.12
H-LC    25.43
H-MC    20.42
M-SC    13.73
M-LC     5.57
M-MC     2.98
L-SC     1.44
L-LC     1.03
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.77,-6.55,41.72
IT,12.45,-20.65,86.44
FINANCE,10.46,-15.38,67.79
MISC,6.93,-18.35,84.15
BANKS,6.45,-10.64,68.84
PAINTS,5.56,-18.55,36.22
ELECTRICAL,5.34,-10.34,49.06
INSURANCE,3.77,-4.93,44.16
AUTO,3.58,-14.78,69.66


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3245808.0,22
XR,1353369.0,14
AR,1196455.0,9
X40,825756.0,10
X40N,632849.0,9
OX40N,558794.0,10
XY25,478050.0,7
SR,258918.0,2
X5K,108253.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3718594.0,29
M-SC,1508275.0,17
H-LC,1263194.0,15
H-MC,1250333.0,15
M-LC,372087.0,4
M-MC,365937.0,2
L-SC,260496.0,3
L-MC,60001.0,1
L-LC,40507.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1226965.0      6
M-SC       XY24       852930.0      7
H-SC       AR         818773.0      5
           XR         778003.0      7
H-MC       XY24       574018.0      4
H-LC       X40        533203.0      5
M-MC       XY24       365937.0      2
H-SC       X40N       310367.0      4
           SR         258918.0      2
           OX40N      254551.0      4
H-LC       X40N       235670.0      3
H-MC       X40        218273.0      4
H-LC       AR         208848.0      2
H-MC       XY25       180731.0      2
L-SC       XR         171971.0      2
M-SC       AR         168834.0      2
M-LC       XY24       151438.0      2
M-SC       OX40N      141404.0      4
           XY25       140779.0      1
           XR         130048.0      2
H-LC       XY25       116033.0      3
M-LC       XR         112396.0      1
           X5K        108253.0      1
H-MC       XR         100950.0      1
H-LC       X200        94920.0      1
L-SC       OX40N       88525.0      1
H-MC       X40N        86812.0      2
H-LC       XY24        74520.0      1
H-MC       OX40N       74314.0      1
M-SC       X40         74280.0      1
H-SC       MH          71017.0      1
L-MC       XR          60001.0      1
L-LC       XY25        40507.0      1
H-MC       MH          15235.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 53.0 seconds
